In [57]:
# %pdb
%matplotlib qt

import numpy as np
import scipy.ndimage as sn
import matplotlib.pyplot as plt
import pathlib as pl
import PIL

In [58]:
ag_path = pl.Path('As Grown/Plain/')
fc_wide_path = pl.Path('Field Cooled/EELS_wide')
fc_close_path = pl.Path('Field Cooled/EELS_close')
fcvc_wide_path = pl.Path('Field Cooled Voltage Conditioned/EELS_wide')
fcvc_close_path = pl.Path('Field Cooled Voltage Conditioned/EELS_close')

ag_names = [name for name in ag_path.iterdir() if 'EELS_' in name.name]
fc_wide_names = [name for name in fc_wide_path.iterdir() if 'EELS_' in name.name]
fc_close_names = [name for name in fc_close_path.iterdir() if 'EELS_' in name.name]
fcvc_wide_names = [name for name in fcvc_wide_path.iterdir() if 'EELS_' in name.name]
fcvc_close_names = [name for name in fcvc_close_path.iterdir() if 'EELS_' in name.name]

ag_image = ag_path / 'EELS.png'
fc_wide_image = fc_wide_path / 'EELS.png'
fc_close_image = fc_close_path / 'EELS.png'
fcvc_wide_image = fcvc_wide_path / 'EELS.png'
fcvc_close_image = fcvc_close_path / 'EELS.png'

images_sizes = [(324.05, 32.86), (128.30, 6.03), (6.67, 2.29), (133.90, 6.67), (20.50, 2.77)] # (x, y) sizes of the images, in nm
images_list = [ag_image, fc_wide_image, fc_close_image, fcvc_wide_image, fcvc_close_image]
eels_names_list = [ag_names, fc_wide_names, fc_close_names, fcvc_wide_names, fcvc_close_names]
labels_list = ["As Grown", "Field Cooled", "Field Cooled", "Field Cooled+Voltage Conditioned", "Field Cooled+Voltage Conditioned"]

keys = ['AG', 'FC_wide', 'FC_close', 'FCVC_wide', 'FCVC_close']

In [59]:
def load_image(path):
    return np.array(PIL.Image.open(path).convert('L'))

def integrate_y(arr, normalize=True):
    integral = np.sum(arr, axis=0)
    
    if normalize:
        sorted_integral = np.sort(integral)
        median_of_top_bins = np.median(sorted_integral[-50:])
        median_of_bottom_bins = np.median(sorted_integral[:50])
        return (integral - median_of_bottom_bins)/(median_of_top_bins-median_of_bottom_bins) # Normalize the integral
    else:
        return integral

def add_to_plot(ax, path):
    colors = get_element_colors()
    linestyles = get_element_linestyles()
    element = get_element(path)
    if element == 'Al':
        return
    ax.plot(integrate_y(load_image(path)),
            linestyle=linestyles[element],
            color=colors[element],
            marker='',
            label=element,
            linewidth=3)
    return

def get_element(path):
    return path.stem.split('_')[-1].capitalize()

def get_element_colors():
    colors = {'Si': 'black',
              'Co': 'midnightblue',
              'Ni': 'darkgreen',
              'Pt': 'grey',
              'Gd': 'darkgoldenrod',
              'O': 'sienna',
              'Al': 'red'}
    return colors

def get_element_linestyles():
    colors = {'Si': '-',
              'Co': '-',
              'Ni': '-',
              'Pt': '-',
              'Gd': '-',
              'O': '-',
              'Al': '-'}
    return colors

def generate_data_structure(normalize=False):
    data = dict()
    for key, image, element_paths, (image_size_x, image_size_y) in zip(keys, images_list, eels_names_list, images_sizes):
        data[key] = dict()
        for element_path in element_paths:
            element = get_element(element_path)
            data[key][element] = integrate_y(load_image(element_path), normalize=normalize)
        data[key]['image'] = image
        data[key]['image_size'] = image_size_x
    return data

In [60]:
def generate_EELS_distribution(tem_image_path, element_image_paths):
    fig, axes = plt.subplots(2,1,figsize=(10,6), sharex=True)
    img = load_image(tem_image_path)
    axes[0].imshow(img, cmap='Greys_r', aspect='auto', interpolation='nearest')
    for path in element_image_paths:
        add_to_plot(axes[1], path)
    axes[1].legend(prop={'size': 20})
    axes[0].set_yticklabels([])
    axes[1].set_yticklabels([])
    axes[1].set_ylabel('Normalized Intensity (a.u.)', size='x-large')
    axes[0].set_xlim([0,img.shape[1]])
    plt.tight_layout()
    return

In [27]:
for image, name, label in zip(images_list, eels_names_list, labels_list):
    generate_EELS_distribution(image, name)
    plt.xlabel(label, size='xx-large')

# generate_EELS_distribution(ag_image, ag_names)
# plt.xlabel("As Grown", size='xx-large')

In [53]:
data = generate_data_structure(normalize=True)
data.keys()

dict_keys(['AG', 'FC_wide', 'FC_close', 'FCVC_wide', 'FCVC_close'])

In [69]:
for sample in ['AG', 'FC_wide', 'FCVC_wide']:
    fig = plt.figure(figsize=(10, 6))
    ax = fig.add_subplot(111)
    for key, value in data[sample].items():
        if key not in ['image', 'image_size', 'Al']:
            ax.plot(np.linspace(0, data[sample]['image_size'], value.shape[0]),
#                     value,
                    sn.convolve(value, np.ones(40)/40, mode='nearest'), 
                    color=get_element_colors()[key], 
                    label=key)
    ax.set_xlim([0, data[sample]['image_size']])
    ax.legend()
    ax.set_title(sample)